---
<img src = '../logo_dh_grupo3.png'>

# <h1><left><ins>Entrenamiento y Evaluación de modelos

## Importación de librerías y bases de datos

In [1]:
sudo pip install jedi
#pip install imblearn

SyntaxError: invalid syntax (<ipython-input-1-96dc65091217>, line 1)

In [2]:
# Importamos las librerias relevantes
import imblearn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn 
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importamos las bases de datos

data_0 = pd.read_csv("base_entrenamiento.csv")

In [4]:
# Cantidad de filas y columnas

data_0.shape

(30360, 22)

La base de datos tiene 30,360 observaciones de 21 columnas.

In [5]:
# Nombre y tipo de columnas, ademas de cantidad de filas no nulas

data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30360 entries, 0 to 30359
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   housing               30360 non-null  int64  
 1   loan                  30360 non-null  int64  
 2   contact               30360 non-null  int64  
 3   campaign              30360 non-null  int64  
 4   emp.var.rate          30360 non-null  float64
 5   cons.price.idx        30360 non-null  float64
 6   cons.conf.idx         30360 non-null  float64
 7   euribor3m             30360 non-null  float64
 8   nr.employed           30360 non-null  float64
 9   y                     30360 non-null  int64  
 10  married               30360 non-null  int64  
 11  month_cat             30360 non-null  int64  
 12  age_cat_(34.0, 44.0]  30360 non-null  int64  
 13  age_cat_(44.0, 69.0]  30360 non-null  int64  
 14  job_cat_1             30360 non-null  int64  
 15  job_cat_2          

Hay 10 variables numéricas y 11 categóricas.

In [6]:
data_1 = data_0.drop(columns = ["campaign"], axis=1)

In [7]:
# Separamos la variable target (y)
X = data_1.drop('y', axis = 1)

y = data_1['y']

In [8]:
# Separamos el dataset en Train y Test:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.30,
                                                   random_state = 42,
                                                   stratify = y)

In [9]:

from sklearn.preprocessing import StandardScaler

sd = StandardScaler()

In [11]:
# Estandarizamos las variables continuas:
variables_continuas = ['emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
                       'nr.employed']

X_train[variables_continuas] = sd.fit_transform(X_train[variables_continuas])

In [12]:
# Transformamos las variables continuas en x_test con la estandarizacion aprendida en el paso anterior (con los datos de entrenamiento)
X_test[variables_continuas] = sd.transform(X_test[variables_continuas])

In [13]:
# Atendemos el problema del desbalanceo de clases con random oversampler:

from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(sampling_strategy = 'minority')

X_train_os, y_train_os = ros.fit_resample(X_train, y_train)

# KNN 

## con GridSearch y Cross Validation

In [14]:
# Realizamos un KNN sin modificar hiperparametros a modo de linea de base:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()


In [15]:
knn.fit(X_train_os, y_train_os)

KNeighborsClassifier()

In [16]:
knn_base_y_pred = knn.predict(X_test)

In [17]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, knn_base_y_pred))

              precision    recall  f1-score   support

           0       0.97      0.86      0.91      8612
           1       0.20      0.58      0.29       496

    accuracy                           0.85      9108
   macro avg       0.58      0.72      0.60      9108
weighted avg       0.93      0.85      0.88      9108



In [18]:
print(confusion_matrix(y_test, knn_base_y_pred))

[[7427 1185]
 [ 206  290]]


Ahora utilizaremos Randomized search para buscar mejores hiperparametros. Si bien Randomized Search no es exahustivo, sino que toma aleatoriamente combinaciones de hiperparametros, es justamente por ello que es más veloz, y sus resultados suelen ser cercanos a los de una busqueda exahustiva. En todo caso, una vez elegido el mejor modelo, se puede correr nuevamente con un GridSearch para obtener los mejores hiperametros posibles. 

In [19]:
#Preparamos un Grid para encontrar el mejor hiperparametro para K
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import RandomizedSearchCV

# Hacemos el grid con una lista de n_neighbours que va del 1 al 20 vecinos mas cercanos.
k_range = list(range(1, 21))
# Tambien evaluamos distintas metricas de distancia:
metrics = ["minkowski", "manhattan"]
param_grid = dict(n_neighbors=k_range, metric=metrics )
#Imprimimos la grilla de parametros:
print(param_grid)

# Indicamos 10 folds para luego hacer cross validation
folds=StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'metric': ['minkowski', 'manhattan']}


Decidimos que los scorings que tienen relevancia practica en este caso son Recall, Precision y, por ende, F1-score ya que este ultimo pone en relacion a los dos primeros. 
Cuando analcemos los resultados veremos en detalle por que seleccionamos estos scorings

In [20]:
# Instanciamos distintas RandomizedSearchCV cada una con un Scoring diferente. 

# F1
rgrid_f1 = RandomizedSearchCV(knn, param_grid, n_iter=15, n_jobs=-2, cv=folds, scoring='f1',random_state=42)

In [26]:
# Recall
rgrid_recall = RandomizedSearchCV(knn, param_grid, n_iter=15, n_jobs=-2, cv=folds, scoring='recall',random_state=42)

In [27]:
# Precision
rgrid_precision = RandomizedSearchCV(knn, param_grid, n_iter=15, n_jobs=-2, cv=folds, scoring='precision',random_state=42)

In [ ]:
# realizamos los RandomizedSearchCV:

#RScv con F1-score
rgrid_f1.fit(X_train_os, y_train_os)

In [24]:
#RScv cpn Recall
rgrid_recall.fit(X_train_os, y_train_os)

KeyboardInterrupt: 

In [ ]:
#RScv con Precision
rgrid_precision.fit(X_train_os, y_train_os)

In [25]:
#Imprimimos los estimadores seleccionados en cada RScv:
print("Mejores parametros para f1:", rgrid_f1.best_params_)
print("Mejores parametros para recall:", rgrid_recall.best_params_)
print("Mejores parametros para precision:", rgrid_precision.best_params_)


AttributeError: 'RandomizedSearchCV' object has no attribute 'best_params_'

In [ ]:
#imprimimos los mejores scores de cada RScv
print("Mejor puntaje de f1-score:", rgrid_f1.best_score_)
print("Mejor puntaje de recall:", rgrid_recall.best_score_)
print("Mejor puntaje de precision:", rgrid_precision.best_score_)

In [ ]:
# Predecimos con y a partir de X_test:
y_pred_rgrid_f1 = rgrid_f1.predict(X_test)

y_pred_rgrid_recall = rgrid_recall.predict(X_test)

y_pred_rgrid_precision = rgrid_precision.predict(X_test)


In [53]:
print (f"y_pred_rgrid_f1:\n",
       classification_report(y_test, y_pred_rgrid_f1))
print (f"y_pred_rgrid_recall:\n",
       classification_report(y_test, y_pred_rgrid_recall))
print (f"y_pred_rgrid_precision:\n",
       classification_report(y_test, y_pred_rgrid_precision))

y_pred_rgrid_f1:
               precision    recall  f1-score   support

           0       0.96      0.88      0.91      8612
           1       0.12      0.29      0.17       496

    accuracy                           0.84      9108
   macro avg       0.54      0.58      0.54      9108
weighted avg       0.91      0.84      0.87      9108

y_pred_rgrid_recall:
               precision    recall  f1-score   support

           0       0.96      0.88      0.91      8612
           1       0.12      0.29      0.17       496

    accuracy                           0.84      9108
   macro avg       0.54      0.58      0.54      9108
weighted avg       0.91      0.84      0.87      9108

y_pred_rgrid_precision:
               precision    recall  f1-score   support

           0       0.95      0.96      0.96      8612
           1       0.16      0.13      0.14       496

    accuracy                           0.91      9108
   macro avg       0.55      0.54      0.55      9108
weighted 

In [54]:
confusion_f1 = confusion_matrix(y_test, y_pred_rgrid_f1)
print(f"y_pred_rgrid_f1 confusion matrix: \n", confusion_f1)
confusion_recall = confusion_matrix(y_test, y_pred_rgrid_recall)
print(f"y_pred_rgrid_recall confusion matrix: \n",confusion_recall)
confusion_precision = confusion_matrix(y_test, y_pred_rgrid_precision)
print(f"y_pred_rgrid_precision confusion matrix: \n",confusion_precision)

y_pred_rgrid_f1 confusion matrix: 
 [[7537 1075]
 [ 352  144]]
y_pred_rgrid_recall confusion matrix: 
 [[7537 1075]
 [ 352  144]]
y_pred_rgrid_precision confusion matrix: 
 [[8269  343]
 [ 432   64]]


Vemos que precision y F1 nos dan la mejor relacion entre la cantidad de llamados y la cantidad de casos de exito, mientras que recall toma la mayor cantidad de casos exitosos aunque a costa de un mayor esfuerzo (costo) en terminos de llamados.

Para ser claros con las metricas de evaluacion, podemos decir que la precisión es la capacidad del clasificador de no etiquetar como positiva una muestra negativa (es decir NO etiquetar como "clientes que van a realizar un deposito a plazo fijo" a aquellos clientes que no tienen intenciones de hacerlo), y la recall es la capacidad del clasificador de encontrar a todos los cliente que SI harian un deposito a plazo fijo dentro del universo de clientes a clasificar. 


Toadavia hay espacios de mejora de lo obtenido a partir del modelo, por ejemplo estableciendo un umbrar (threshold) relacionado con la probabilidad de que una observacion perteneszca a una clase o a otra. Todas las observaciones son analizadas y el modelo establece para cada observacion una probablidad (un valor entre 0 y 1) de que esa observacion sea caso de exito (en este caso seria que el cliente haga un deposito a plazo fijo). 

Por default el modelo cailifica como 0 a todos aquellos clientes cuya probabilidad de exito sea menor a 0.5 y como 1(exito) a todos aquellos clientes cuya probabilidad de exito sea mayor a 0.5. Esto puede ser redefinido a voluntad y encontrar estrategias que busquen encontrar el mejor equilibrio entre una serie de errores posibles, a saber: 

- Se puede establecer un parametro mas duro (ej: probabilidad minima de 0.8 de probabilidad para ser considerado 1)
En este caso es mas "dificil" que un cliente sea catalogado como 1, por lo que solo aquellos clientes que "muy probablemente" adquieran el credito seran seleccionados. Esto establece una mejor relacion entre la cantidad de llamados a realizarse y la cantidad de casos de exito. Sin embargo, justamente porque es tan dificil que un cliente sea catalogaso como 1, no son clasificados como potenciales depositantes algunos clientes que si hubieran hecho el deposito a plazo fijo. Es decir, que se pierden algunos plazos fijos, pero se prodice un ahorro en el costo y duracion de la campaña, sin mencionar que no se importuna a clientes que no tienen la intencion, o la posibilidad de hacer depositos a plazo fijo. 

- O se puede establecer un parametro mas laxo (ej: probabilidad minima de 0.3 para ser considerado 1)
Esta seria la situacion inversa, se prioriza captar a todos los posibles clientes que harian un deposito plazo fijo a costa de clasificar como exitosos a una gran cantidad de clientes que no van a realizar un plazo fijo, esto aunmenta el costo de la campaña y probablemente su duracion, pero tiene la ventaja de que el resultado final sera de mas plazos fijos. 

Elegir como regular este umbral dependera de encontrar la mejor relacion entre los costos asosciados a la campaña y de los beneficios para el banco obtenidos con cada nuevo plazo fijo. 

In [55]:
# realizasmos un predict_proba para obtener las probabilidades de exito de cada cliente
y_pred_proba_f1 = rgrid_f1.predict_proba(X_test)

y_pred_proba_recall = rgrid_recall.predict_proba(X_test)

y_pred_proba_precision = rgrid_precision.predict_proba(X_test)

In [56]:
#Binarizamos a unos y ceros estableciendo el umbral en funcion del criterio elegido. 
from sklearn.preprocessing import binarize
y_proba_f1 = binarize(y_pred_proba_f1, threshold=0.5)[:,1]

print (f"y_proba_f1:\n",
       classification_report(y_test, y_proba_f1))
confusion_f1_proba = confusion_matrix(y_test, y_proba_f1)
print(f"y_proba_f1 confusion matrix: \n",confusion_f1_proba)

y_proba_f1:
               precision    recall  f1-score   support

           0       0.96      0.88      0.91      8612
           1       0.12      0.29      0.17       496

    accuracy                           0.84      9108
   macro avg       0.54      0.58      0.54      9108
weighted avg       0.91      0.84      0.87      9108

y_proba_f1 confusion matrix: 
 [[7537 1075]
 [ 352  144]]


In [57]:
#Probamos lo mismo con Recall:
y_proba_recall = binarize(y_pred_proba_recall, threshold=0.7)[:,1]

print (f"y_proba_recall:\n",
       classification_report(y_test, y_proba_recall))
confusion_recall_proba = confusion_matrix(y_test, y_proba_recall)
print(f"y_proba_recall confusion matrix: \n",confusion_recall_proba)

y_proba_recall:
               precision    recall  f1-score   support

           0       0.95      0.95      0.95      8612
           1       0.15      0.16      0.15       496

    accuracy                           0.91      9108
   macro avg       0.55      0.55      0.55      9108
weighted avg       0.91      0.91      0.91      9108

y_proba_recall confusion matrix: 
 [[8181  431]
 [ 419   77]]


In [58]:
# Y con Precision:
y_proba_precision = binarize(y_pred_proba_precision, threshold=0.4)[:,1]

print (f"y_proba_precision:\n",
       classification_report(y_test, y_proba_precision))

confusion_precision_proba = confusion_matrix(y_test, y_proba_precision)
print(f"y_proba_precision confusion matrix: \n",confusion_precision_proba)

y_proba_precision:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94      8612
           1       0.14      0.19      0.16       496

    accuracy                           0.89      9108
   macro avg       0.54      0.56      0.55      9108
weighted avg       0.91      0.89      0.90      9108

y_proba_precision confusion matrix: 
 [[7997  615]
 [ 400   96]]


In [59]:
total_llamados_test = len(y_test)
porcentaje_test = (len(y_test)*100)/len(y_test)
exitos_en_test = y_test.sum()
llam_x_exito_test = (len(y_test)/(y_test.sum())).round(1)
porcentaje_del_total_en_test= ((y_test.sum()*100)/y_test.sum())

total_llamados_knn_f1 = (confusion_f1_proba[0,1]+confusion_f1_proba[1,1])
porcentaje_knn_f1 = (total_llamados_knn_f1*100)/len(y_test)
exitos_knn_f1= confusion_f1_proba[1,1]
llam_x_exito_en_knn_f1 = ((confusion_f1_proba[0,1]+confusion_f1_proba[1,1])/confusion_f1_proba[1,1]).round(1)
porcentaje_del_total_en_knn_f1=  ((confusion_f1_proba[1,1]*100)/y_test.sum()).round(1)

total_llamados_knn_recall = (confusion_recall_proba[0,1]+confusion_recall_proba[1,1])
porcentaje_knn_recall = (total_llamados_knn_recall*100)/len(y_test)
exitos_knn_recall = confusion_recall_proba[1,1]
llam_x_exito_en_knn_recall  = ((confusion_recall_proba[0,1]+confusion_recall_proba[1,1])/confusion_recall_proba[1,1]).round(1)
porcentaje_del_total_en_knn_recall = ((confusion_recall_proba[1,1]*100)/y_test.sum()).round(1)


In [60]:
est = pd.DataFrame()
est["modelo"]= ["y_test-fuerza_bruta-","knn_recall","knn_f1"]
est["total_llamados"] = [total_llamados_test, total_llamados_knn_recall, total_llamados_knn_f1]
est["%_del_total_de_llamados"] = [porcentaje_test, porcentaje_knn_recall, porcentaje_knn_f1]
est["exitos"] = [exitos_en_test, exitos_knn_recall, exitos_knn_f1]
est["llamados_x_cada_exito"] = [llam_x_exito_test, llam_x_exito_en_knn_recall, llam_x_exito_en_knn_f1]
est["recall"] = [porcentaje_del_total_en_test, porcentaje_del_total_en_knn_recall, porcentaje_del_total_en_knn_f1]
est.sort_values("recall", ascending=False, ignore_index=True)

,modelo,total_llamados,%_del_total_de_llamados,exitos,llamados_x_cada_exito,recall
0,y_test-fuerza_bruta-,9108,100.000000,496,18.4,100.0
1,knn_f1,1219,13.383838,144,8.5,29.0
2,knn_recall,508,5.577514,77,6.6,15.5


Podemos decir que el mejor modelo de KNN es KNN_RECALL que captura el 56.7 de los casos de exitos con un costo que es proporcional al 15,3% del costo de la campaña anterior (con metodo de fuerza bruta, es decir, llamar a todos los clientes). 

Se puede reducir significativamente el esfuerzo de campaña y aun asi obtener resultados. Esto se traduce en que se pueden hacer campañas mas cortas pre seleccionando solo aquellos clientes del banco que tengan altas probabilidades de exito. Disminuyendo la duracion y el costo de la campaña.

Hemos encontrado en estos hiperparametros cierto equilibrio entre efectividad y eficacia. Podriamos elegir otros hiperparametoros que resultarian en un modelo que identificaria mejor a los casos de exito (es decir no dejar afuera ningun cliente que hubiera aceptado hacer un deposito a plazo fijo), pero aumentarian los llamados disminuyendo la efecacia del sistema. 

Hasta que punto conviene cambiar llamados por plazos fijos es un criterio que debiera definir el banco en funcion de costos y beneficios. Es decir si prefiere invertir en mas llamados para captar la mayor catidad de plazos fijos posibles, o buscar algun equilibrio entre minimizar la inversion en la campaña y aumentar su eficiencia. 

De todos modos todavia quedan modelos mas complejos para ser evaluados como XGBoost, RandomForest, etc. que podrían arrojar mejores resultados..



In [61]:
#Guardamos los modelos entrenados en archivos picke para su posterior acceso:

#import pickle
#with open('knn-recall','wb') as f:
 #   pickle.dump(rgrid_recall,f)
  #  print('dump sucess! =)')


dump sucess! =)
